In [ ]:
# Due to tweepy daily request limit the following code coundn't be executed.
# But they where tested on a smaller request with few user and works as expected.

import tweepy
import json
import csv  
# API keys
api_key = "f2yvgnv2OA65h4FAMQXHopMqz"
api_secrets = "CQIDIcxH8pjwUyve8EB7uFBJSHA6fWtEQhcMpXZKfRjo0srle7"
access_token = "1565754535993217025-8sEPfWnXRimhTJBXqMIsm73GWS1BC9"
access_secret = "ogH7SoQkKSowFXBUx51U6qZkSu9JliQ9fmU6KCJFA4LHl"
 
# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key,api_secrets)
auth.set_access_token(access_token,access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# Data extraction from twitter.
# Crawl through 20 random follower of elonmusk and continue crawling for three layer
# This will give us around 160000 userid list
userIdList = open("userId.txt", "a",encoding="utf-8")
for followers in tweepy.Cursor(api.get_followers, screen_name = 'elonmusk').items(20):
    user_Id = followers._json.get('id_str') + '\n'
    userIdList.write(user_Id)
    for firstCrawl in tweepy.Cursor(api.get_followers, user_id = user_Id).items(20):
        user_Id_1 = firstCrawl._json.get('id_str') + '\n'
        userIdList.write(user_Id_1)
        for secondCrawl in tweepy.Cursor(api.get_followers, user_id = user_Id_1).items(20):
            user_Id_2 = firstCrawl._json.get('id_str') + '\n'
            userIdList.write(user_Id_2)
            for thirdCrawl in tweepy.Cursor(api.get_followers, user_id = user_Id_2).items(20):
                user_Id_3 = firstCrawl._json.get('id_str') + '\n'
                userIdList.write(user_Id_3)
userIdList.close()

# Reading the final user id list and getting the their features with user description, favorites, tweets etc.

# Get user info, tweets, favorites and write them into a csv file.
finalUserId = open('userId.txt', mode='r')
header = ['user_info', 'favorites', 'hashTags', 'tweetText']
with open('twitterData.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for userId in finalUserId:ap
        userInfo = api.get_user(user_id=userId)
        for favorites in tweepy.Cursor(api.get_favorites, user_id=userId).items():
            favoritesTweet = favorites._json
        for tweet in tweepy.Cursor(api.user_timeline, user_id=userId, include_rts=True, tweet_mode= "extended").items():
            hashTagMaps = tweet._json.get('entities').get('hashtags')
            tweetText = tweet._json.get('full_text')
        data = [userInfo,favoritesTweet,hashTagMaps,tweetText]
        writer.writerow(data)
        
# Create edge list of all the user extracted
header_edgeList = ['from', 'to']
with open('twitterUserEdgeList.csv', 'w', encoding='UTF8') as f1:
    writer = csv.writer(f1)
    writer.writerow(header_edgeList)
    for userId in finalUserId:
        for userId_2 in finalUserId:
            createEdge = api.exists_friendship(userId,userId_2) # if this returns true, create edge list of user id as graph node.
            if (createEdge):
                data = [userId, userId_2]
                writer.writerow(data)


# With this we will have the required data for our analysis of homophily in social network.